# Тестирование Spell Checker с использованием дополнительных признаков

В этом ноутбуке будет произведено тестирование модели, в которой в candidate scorer используются модели на основе множества признаков помимо BERT.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gc
import sys
import os
import json
import pickle
import re
from string import punctuation
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd
from transformers import BertForMaskedLM, BertTokenizer, BertConfig

from deeppavlov.core.data.simple_vocab import SimpleVocabulary

import kenlm
from sacremoses import MosesTokenizer, MosesDetokenizer

from src.models.SpellChecker import *
from src.models.BertScorer.bert_scorer_correction import (
    BertScorerCorrection
)

from sklearn.svm import LinearSVC

from IPython.display import display
from tqdm.notebook import tqdm

In [ ]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
MODEL_PATH = os.path.join(PROJECT_PATH, 'models')

## Инициализация

Начнем с того, что инициализируем все необходимые компоненты модели. Параллельно так же будет описана роль каждого компонента в системе.

In [ ]:
raw_tokenizer = MosesTokenizer(lang='ru')
raw_detokenizer = MosesDetokenizer(lang='ru')
tokenizer = lambda x: raw_tokenizer.tokenize(x, escape=False)
detokenizer = lambda x: raw_detokenizer.detokenize(x)

In [ ]:
vocab_path = os.path.join(DATA_PATH, 'external', 'russian_words', 
                          'russian_words_vocab.dict')
vocab = SimpleVocabulary(load_path=vocab_path, save_path=vocab_path)
handcode_table_path = os.path.join(DATA_PATH, 'processed', 'handcode_table', 
                                   'table.json')
with open(handcode_table_path, 'r') as inf:
    handcode_table = json.load(inf)
candidate_generator = CandidateGenerator(
    words=vocab.keys(), handcode_table=handcode_table, max_distance=1
)

In [ ]:
model_left_right = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'left_right_3_100.arpa.binary')
)
model_right_left = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'right_left_3_100.arpa.binary')
)
margin_border = np.log(2.5)
position_selector = KenlmMarginPositionSelector(
    model_left_right, model_right_left, margin_border=margin_border
)

In [ ]:
BERT_PATH = os.path.join(MODEL_PATH, 'conversational_rubert')
config = BertConfig.from_json_file(
    os.path.join(BERT_PATH, 'bert_config.json')
)
model = BertForMaskedLM.from_pretrained(
    os.path.join(BERT_PATH, 'pytorch_model.bin'),
    config=config
)
bert_tokenizer = BertTokenizer(os.path.join(BERT_PATH, 'vocab.txt'))
bert_scorer_correction = BertScorerCorrection(model, bert_tokenizer)
agg_subtoken_func = 'mean'
bert_scorer = BertScorer(
    bert_scorer_correction, agg_subtoken_func
)

with open(os.path.join(MODEL_PATH, 'candidate_scorer', 'svm.bin'), 'rb') as inf:
    svm_model = pickle.load(inf)

svm_scorer = SVMScorer(svm_model, bert_scorer=bert_scorer)
candidate_scorer = CandidateScorer(svm_scorer)

In [ ]:
# максимальное количество итераций
max_it = 5

spellchecker = IterativeSpellChecker(
    candidate_generator,
    position_selector,
    candidate_scorer,
    tokenizer,
    detokenizer,
    ignore_titles=True,
    max_it=max_it,
    combine_tokens=True
)

## Тестирование

### Обучающая выборка

In [ ]:
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'train_source.txt'), 
    'r'
) as inf:
    sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'train_corrected.txt'), 
    'r'
) as inf:
    true_sentences = inf.readlines()

Запустим наш spell checker, подавая ему предложения батчами размера `batch_size`.

In [ ]:
batch_size = 5
sentences_corrected = []
num_batches = int(np.ceil(len(sentences) / batch_size))

for i in tqdm(range(num_batches)):
    cur_sentences = sentences[i*batch_size:(i+1)*batch_size]
    sentences_corrected += spellchecker(cur_sentences)

Запишем результаты файл.

In [ ]:
!mkdir -p ../data/processed/results_svm/

In [ ]:
with open(os.path.join(DATA_PATH, 'processed', 'results_svm', 'train.txt'), 'w') as ouf:
    ouf.writelines([sentence + '\n' for sentence in sentences_corrected])

Выполним скрит для измерения качества.

In [ ]:
!python ../src/evaluation/spell_ru_eval/evaluate.py -d ../data/processed/results_svm/diffs_train.txt ../data/external/spell_ru_eval/train_source.txt ../data/external/spell_ru_eval/train_corrected.txt ../data/processed/results_svm/train.txt | tail -n 4

**Ranking SVM**

* True Positive: $1311$
* Внесенных исправлений: $1451$ 
* Требуемых исправления: $1727$ 
* Precision: $90.35$, доверительный интервал: $(88.83, 91.87)$
* Recall: $75.91$, доверительный интервал: $(74.63, 77.19)$
* FMeasure: $82.50$, доверительный интервал: $(81.11, 83.90)$

**Ranking SVM (с объединением токенов)**

* True Positive: $1331$
* Внесенных исправлений: $1479$ 
* Требуемых исправления: $1727$ 
* Precision: $89.99$, доверительный интервал: $(88.47, 91.52)$
* Recall: $77.07$, доверительный интервал: $(75.76, 78.38)$
* FMeasure: $83.03$, доверительный интервал: $(81.62, 84.44)$

**Ranking SVM (без BERT)**

* True Positive: $1277$
* Внесенных исправлений: $1428$ 
* Требуемых исправления: $1727$ 
* Precision: $87.83$, доверительный интервал: $(87.83, 91.02)$
* Recall: $73.94$, доверительный интервал: $(72.62, 75.26)$
* FMeasure: $80.95$, доверительный интервал: $(79.51, 82.40)$

**CatBoost**

* True Positive: $1358$
* Внесенных исправлений: $1458$ 
* Требуемых исправления: $1727$ 
* Precision: $93.19$, доверительный интервал: $(91.84, 94.44)$
* Recall: $78.63$, доверительный интервал: $(77.54, 79.73)$
* FMeasure: $85.27$, доверительный интервал: $(84.08, 86.47)$

**CatBoost (с объединением токенов)**

* True Positive: $1385$
* Внесенных исправлений: $1495$ 
* Требуемых исправления: $1727$ 
* Precision: $92.64$, доверительный интервал: $(91.32, 93.97)$
* Recall: $80.20$, доверительный интервал: $(79.05, 81.34)$
* FMeasure: $85.95$, доверительный интервал: $(84.74, 87.20)$

**CatBoost (без BERT)**

* True Positive: $1324$
* Внесенных исправлений: $1454$ 
* Требуемых исправления: $1727$ 
* Precision: $91.06$, доверительный интервал: $(89.59, 92.53)$
* Recall: $76.66$, доверительный интервал: $(75.43, 77.90)$
* FMeasure: $83.24$, доверительный интервал: $(81.90, 84.59)$

### Тестовая выборка

In [ ]:
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'test_source.txt'), 
    'r'
) as inf:
    sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'test_corrected.txt'), 
    'r'
) as inf:
    true_sentences = inf.readlines()

Запустим наш spell checker, подавая ему предложения батчами размера `batch_size`.

In [ ]:
batch_size = 5
sentences_corrected = []
num_batches = int(np.ceil(len(sentences) / batch_size))

for i in tqdm(range(num_batches)):
    cur_sentences = sentences[i*batch_size:(i+1)*batch_size]
    sentences_corrected += spellchecker(cur_sentences)

Запишем результаты файл.

In [ ]:
with open(os.path.join(DATA_PATH, 'processed', 'results_svm', 'test.txt'), 'w') as ouf:
    ouf.writelines([sentence + '\n' for sentence in sentences_corrected])

Выполним скрит для измерения качества.

In [ ]:
!python ../src/evaluation/spell_ru_eval/evaluate.py -d ../data/processed/results_svm/diffs_test.txt ../data/external/spell_ru_eval/test_source.txt ../data/external/spell_ru_eval/test_corrected.txt ../data/processed/results_svm/test.txt | tail -n 4

**Ranking SVM**

* True Positive: $1393$
* Внесенных исправлений: $1596$ 
* Требуемых исправления: $1976$ 
* Precision: $87.28$, доверительный интервал: $(85.66, 88.91)$
* Recall: $70.50$, доверительный интервал: $(69.18, 71.81)$
* FMeasure: $78.00$, доверительный интервал: $(76.54, 79.45)$

**Ranking SVM (с объединением токенов)**

* True Positive: $1438$
* Внесенных исправлений: $1648$ 
* Требуемых исправления: $1976$ 
* Precision: $87.26$, доверительный интервал: $(85.64, 88.87)$
* Recall: $72.77$, доверительный интервал: $(71.42, 74.12)$
* FMeasure: $79.36$, доверительный интервал: $(77.89, 80.83)$

**Ranking SVM (без BERT)**

* True Positive: $1348$
* Внесенных исправлений: $1566$ 
* Требуемых исправления: $1976$
* Precision: $86.08$, доверительный интервал: $(84.37, 87.79)$
* Recall: $68.22$, доверительный интервал: $(66.87, 69.57)$
* FMeasure: $76.12$, доверительный интервал: $(74.61, 77.62)$

**CatBoost**

* True Positive: $1375$
* Внесенных исправлений: $1578$ 
* Требуемых исправления: $1976$
* Precision: $87.14$, доверительный интервал: $(85.48, 88.79)$
* Recall: $69.59$, доверительный интервал: $(68.27, 70.90)$
* FMeasure: $77.38$, доверительный интервал: $(75.91, 78.84)$

**CatBoost (с объединением токенов)**

* True Positive: $1427$
* Внесенных исправлений: $1640$ 
* Требуемых исправления: $1976$
* Precision: $87.01$, доверительный интервал: $(85.38, 88.64)$
* Recall: $72.22$, доверительный интервал: $(70.86, 73.57)$
* FMeasure: $78.93$, доверительный интервал: $(77.45, 80.41)$

**CatBoost (без BERT)**

* True Positive: $1346$
* Внесенных исправлений: $1569$ 
* Требуемых исправления: $1976$
* Precision: $85.79$, доверительный интервал: $(84.06, 87.51)$
* Recall: $68.12$, доверительный интервал: $(66.75, 69.49)$
* FMeasure: $75.94$, доверительный интервал: $(74.41, 77.47)$

## Выводы

1. Результаты удалось заметно улучшить и достичь SOTA.
2. CatBoost дал улучшение на обучающей выборке, но на тестовой лучше оказалась модель на SVM.
3. Модели без признаков BERT показывают результат на $\approx 1.5\%$ хуже.
4. Добавление объединения токенов улучшает результат на $\approx 1\%$.